In [2]:
!pip install mediapipe tensorflow opencv-python pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.6 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-decision-forests 1.10.0 requires tens

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Define dataset path (adjust based on Kaggle dataset mount location)
DATASET_PATH = "/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train"

# Image parameters
IMG_SIZE = 64  # Resize images to 64x64
BATCH_SIZE = 32

# Data augmentation & normalization
datagen = ImageDataGenerator(
    rescale=1./255,    # Normalize pixel values
    validation_split=0.2  # 20% for validation
)

# Load training data
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get class names
class_names = list(train_data.class_indices.keys())
print("Classes:", class_names)


Found 178472 images belonging to 29 classes.
Found 44602 images belonging to 29 classes.
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [9]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define dataset path (adjust based on Kaggle dataset location)
DATASET_PATH = "/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train"

# Image parameters
IMG_SIZE = 64  # Resize images
BATCH_SIZE = 32

# Define Image Data Generator with train-validation split
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    validation_split=0.2,  # 20% for validation
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get class names
class_names = list(train_generator.class_indices.keys())
print("Classes:", class_names)

# Load MobileNetV2 as base model
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze base model layers

# Add classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce dimensions
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_layer = Dense(len(class_names), activation='softmax')(x)  # Output layer for ASL classes

# Define the final model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint("best_model.keras", monitor="val_accuracy", save_best_only=True, mode="max")

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # Reduce to 10 epochs to save time
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Save final model
model.save("asl_trained_model.keras")


Found 178472 images belonging to 29 classes.
Found 44602 images belonging to 29 classes.
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


<ipython-input-9-e742e28dd4f5>:54: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")


Epoch 1/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 636s 113ms/step - accuracy: 0.2695 - loss: 2.5207 - val_accuracy: 0.3979 - val_loss: 2.1130 - learning_rate: 0.0010
Epoch 2/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 640s 115ms/step - accuracy: 0.4009 - loss: 1.9790 - val_accuracy: 0.4174 - val_loss: 2.0702 - learning_rate: 0.0010
Epoch 3/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 642s 115ms/step - accuracy: 0.4251 - loss: 1.8914 - val_accuracy: 0.4266 - val_loss: 2.0274 - learning_rate: 0.0010
Epoch 4/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 727s 130ms/step - accuracy: 0.4326 - loss: 1.8626 - val_accuracy: 0.4379 - val_loss: 2.0088 - learning_rate: 0.0010
Epoch 5/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 771s 138ms/step - accuracy: 0.4409 - loss: 1.8359 - val_accuracy: 0.4408 - val_loss: 2.0019 - learning_rate: 0.0010
Epoch 6/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 766s 137ms/step - accuracy: 0.4482 - loss: 1.8098 - val_accuracy: 0.4495 - val_loss: 1.9787 - learning_rate: 0.0010
Epoch 7/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 788s 141ms

In [10]:
# Import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Set dataset path (Kaggle dataset location)
DATASET_PATH = "/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train"

# Image parameters
IMG_SIZE = 64  # Resize images to 64x64
BATCH_SIZE = 32

# Data Augmentation & Normalization
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% data for validation
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load training data
train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# Load validation data
val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Get class names (labels)
class_names = list(train_data.class_indices.keys())
print("Classes:", class_names)

# Load Pre-trained MobileNetV2 Model (Feature Extractor)
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze the whole MobileNetV2 initially

# Build Custom Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)  # Normalize activations
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
output_layer = Dense(len(class_names), activation='softmax')(x)

# Create Final Model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Model Summary
model.summary()

# Define Callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6
)

checkpoint = ModelCheckpoint(
    "best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max"
)

# Train Model (Phase 1 - Feature Extraction)
history = model.fit(
    train_data,
    epochs=5,  # Initial 5 epochs
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# **Fine-Tune Model (Unfreeze Last 20 Layers)**
for layer in base_model.layers[-20:]:  
    layer.trainable = True

# Recompile model with lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train Model Again (Phase 2 - Fine-Tuning)
history_finetune = model.fit(
    train_data,
    epochs=10,  # Train 10 more epochs after unfreezing
    validation_data=val_data,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Save Final Model
model.save("asl_model_final.keras")

print("🎉 Training Complete! Model Saved as 'asl_model_final.keras'")


Found 178472 images belonging to 29 classes.
Found 44602 images belonging to 29 classes.
Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


<ipython-input-10-fa678c5d6d41>:54: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 64, 64, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 32, 32, 32)     │            864 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 32, 32, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 32, 32, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 32, 32, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 32, 32, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 32, 32, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 32, 32, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 32, 32, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 32, 32, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 32, 32, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 33, 33, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 16, 16, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 3,055,709 (11.66 MB)

 Trainable params: 796,189 (3.04 MB)

 Non-trainable params: 2,259,520 (8.62 MB)

Epoch 1/5
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 671s 119ms/step - accuracy: 0.3035 - loss: 2.4698 - val_accuracy: 0.4171 - val_loss: 2.0495 - learning_rate: 0.0010
Epoch 2/5
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 784s 141ms/step - accuracy: 0.4468 - loss: 1.8232 - val_accuracy: 0.4428 - val_loss: 1.9706 - learning_rate: 0.0010
Epoch 3/5
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 815s 146ms/step - accuracy: 0.4783 - loss: 1.7085 - val_accuracy: 0.4598 - val_loss: 1.9252 - learning_rate: 0.0010
Epoch 4/5
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 819s 147ms/step - accuracy: 0.4997 - loss: 1.6335 - val_accuracy: 0.4786 - val_loss: 1.8671 - learning_rate: 0.0010
Epoch 5/5
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 823s 147ms/step - accuracy: 0.5129 - loss: 1.5893 - val_accuracy: 0.4838 - val_loss: 1.8640 - learning_rate: 0.0010
Epoch 1/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 856s 151ms/step - accuracy: 0.4618 - loss: 1.8291 - val_accuracy: 0.6120 - val_loss: 1.4380 - learning_rate: 1.0000e-04
Epoch 2/10
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 830s 149ms/

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Define constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10  # Increase as needed
LEARNING_RATE = 1e-5  # Lower LR for fine-tuning
DATA_DIR = "/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_train"  # Update path

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% train, 20% validation
)

# Load train & validation data
train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Load MobileNetV2 with pre-trained weights
base_model = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze base layers

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)  # Reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(29, activation='softmax')(x)  # 27 classes (excluding 'space' and 'del')

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint("asl_model_finetuned.keras", monitor='val_accuracy', save_best_only=True, verbose=1)

# Initial Training (Before Unfreezing Layers)
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5,  # Initial training
    callbacks=[reduce_lr, checkpoint]
)

# Unfreeze some layers for fine-tuning
for layer in base_model.layers[-30:]:  # Unfreezing last 30 layers
    layer.trainable = True

# Compile again with lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

# Train again with fine-tuning
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  # Fine-tuning epochs
    callbacks=[reduce_lr, checkpoint]
)

# Save final model
model.save("asl_model_final.keras")

print("🎉 Fine-tuning complete! Model saved.")


Found 178472 images belonging to 29 classes.
Found 44602 images belonging to 29 classes.
Epoch 1/5
5577/5578 ━━━━━━━━━━━━━━━━━━━━ 0s 368ms/step - accuracy: 0.1349 - loss: 3.1406
Epoch 1: val_accuracy improved from -inf to 0.52590, saving model to asl_model_finetuned.keras
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 2632s 470ms/step - accuracy: 0.1349 - loss: 3.1405 - val_accuracy: 0.5259 - val_loss: 1.8883 - learning_rate: 1.0000e-05
Epoch 2/5
5577/5578 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - accuracy: 0.4685 - loss: 1.8144
Epoch 2: val_accuracy improved from 0.52590 to 0.61881, saving model to asl_model_finetuned.keras
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 2612s 468ms/step - accuracy: 0.4685 - loss: 1.8143 - val_accuracy: 0.6188 - val_loss: 1.4451 - learning_rate: 1.0000e-05
Epoch 3/5
5577/5578 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - accuracy: 0.5878 - loss: 1.3607
Epoch 3: val_accuracy improved from 0.61881 to 0.65658, saving model to asl_model_finetuned.keras
5578/5578 ━━━━━━━━━━━━━━━━━━━━ 2619s 469ms/step -

KeyboardInterrupt: 

In [11]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load Model
model = load_model("asl_model_final.keras")

# Load & Preprocess Test Image
img_path = "/kaggle/input/aslamerican-sign-language-aplhabet-dataset/ASL_Alphabet_Dataset/asl_alphabet_test/H_test.jpg"
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img) / 255.0  # Normalize
img_array = np.expand_dims(img_array, axis=0)

# Predict
predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]
print("Predicted ASL Letter:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted ASL Letter: H
